# Exploring and Clustering Toronto Neighborhoods

In this notebook we will explore and cluster Toronto neighborhoods

## Contents
1. Reading Data into DataFrame
1. Merging Neighborhoods DataFrames with the coordinates DataFrame
1. Explore Neighbourhoods

### (1) Data Wrangling

#### Import required analysis packages

In [1]:
import pandas as pd
import numpy as np

# Set the url where the data is located
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# read all the tables from the 
tables = pd.read_html(url)


#### Retrieving the table

The Data is contained in the first table therefore we ignore the other tables

In [2]:
# we only need the data from the first table
toronto_neighborhoods_df = tables[0]
toronto_neighborhoods_df = toronto_neighborhoods_df[toronto_neighborhoods_df.Borough != 'Not assigned']

#### Format the DataFrame into the desired Structure

In [3]:
# The data we are looking for has unique Postcodes so first we select those and keep them in a numpy array
# Then we create lists which will contain the data corresponding to the unique Postcodes
filtered_postcodes_df = toronto_neighborhoods_df['Postcode'].unique()
neighbourhood = []
borough = []

# basically filling the lists with data that corresponds to the Postcodes unique values
for neigh in filtered_postcodes_df:
    temp_df = toronto_neighborhoods_df[toronto_neighborhoods_df['Postcode'] == neigh]
    neighbourhoods = ', '.join(temp_df.Neighbourhood)
    neighbourhood.append(neighbourhoods)
    borough.append(''.join(temp_df.Borough.unique().tolist()))
    
    
# Creating the dataframe that we want from the lists that was created
neigh_na = toronto_neighborhoods_df.drop(['Neighbourhood'], axis=1)
merged_toronto_neighbourhood_df = pd.DataFrame(filtered_postcodes_df, columns=['Postcode']).join(pd.DataFrame(borough, columns=['Borough'])).join(pd.DataFrame(neighbourhood, columns=['Neighbourhood']))


# Replacing the "Not assigned" values in the Neighbourhood with the values contained in the Borough column
for i, neigh, bor in zip(merged_toronto_neighbourhood_df.index,  merged_toronto_neighbourhood_df['Neighbourhood'], merged_toronto_neighbourhood_df['Borough']):
    if neigh == 'Not assigned':
        merged_toronto_neighbourhood_df.loc[i, ['Neighbourhood']] = bor

# Preview the desired DataFrame  
merged_toronto_neighbourhood_df.head(10)
    

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Desired DataFrame Shape

In [4]:
merged_toronto_neighbourhood_df.shape

(103, 3)

### (2) Merging Neighborhoods DataFrames with the coordinates DataFrame

#### Get the Coordinates Data and input into a DataFrame

In [5]:
# Get the data from the url and read the data into the DataFrame
url='https://cocl.us/Geospatial_data'
toronto_coordinates_df=pd.read_csv(url)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the dataframes to include the coordinates in the neighbood DataFrame

In [6]:
# merge the dataframes
toronto_coordinates_df = toronto_coordinates_df.rename(columns = {'Postal Code': 'Postcode'})
toronto_coordinated_neighborhoods = merged_toronto_neighbourhood_df.merge(toronto_coordinates_df, how='inner', on='Postcode')
toronto_coordinated_neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Install Folium package and import 

This package will help in creating the map

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.



### (3) Explore the Neighborhoods

We will do this by Visualizing the data on a map

#### Get the Coordinates for Toronto

For the sake of this exercise, we will assume that we are at "Parkwoods" Neighborhood and use it's corresponding coordinates as a referece point to create the map

In [8]:
# Coordinates for Parkwoods neighborhood
latitude = 43.753259
longitude = -79.329656

#### Generate the map

In [12]:
toronto_neighborhoods_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around Parkwoods

# # We add a Green marker to represent where we are (Parkwoods)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Parkwoods',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(toronto_neighborhoods_map)

# We add the rest of the postal codes on the map
for lat, lng, label in zip(toronto_coordinated_neighborhoods['Latitude'], toronto_coordinated_neighborhoods['Longitude'], toronto_coordinated_neighborhoods['Postcode']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=str(label),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(toronto_neighborhoods_map)

# # display map
toronto_neighborhoods_map